In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive/Major\ Project

/content/drive/.shortcut-targets-by-id/1ip9U1PHHqJrfXpdXp5tEPrx29H2d9jH6/Major Project


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import models
import os
import numpy as np
from matplotlib import pyplot as plt
import copy
import PIL
from torchvision import models
from PIL import Image
import pandas as pd

In [ ]:
# check GPU availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
# Data loading (Modiflication required)
from torchvision import transforms,datasets
transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
data_dir = 'ImageDatasets/BnWDataset2/'
image_dataset = datasets.ImageFolder(data_dir,transform)
total_count = len(image_dataset)
train_count = int(0.8 * total_count)
test_count = total_count - train_count
train_dataset, test_dataset = torch.utils.data.random_split(image_dataset, [train_count, test_count])

dataloaders = {'train': torch.utils.data.DataLoader(train_dataset, batch_size=4,
                                             shuffle=True, num_workers=4),
               'val': torch.utils.data.DataLoader(test_dataset, batch_size=4,
                                             shuffle=True, num_workers=4)}
dataset_sizes = {'train': len(train_dataset),'val':len(test_dataset)}
class_names = image_dataset.classes

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# Downloading VGG network
vgg16 = models.vgg16(pretrained=True)
vgg16.to(device)
print(vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# Freezing convolution weights
for param in vgg16.features.parameters():
    param.requires_grad = False
# change the number of classes 
vgg16.classifier = nn.Sequential(
    nn.Linear(in_features=25088, out_features=4096, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=4096, out_features=4096, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=4096, out_features=len(class_names), bias=True)
)
Net = vgg16

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    try:
      for epoch in range(num_epochs):
        
          print('Epoch {}/{}'.format(epoch, num_epochs - 1))
          print('-' * 10)

          for phase in ['train', 'val']:
              if phase == 'train':
                  scheduler.step()
                  model.train()  # Set model to training mode
              else:
                  model.eval()   # Set model to evaluate mode

              running_loss = 0.0
              running_corrects = 0

              # Iterate over data.
              for inputs, labels in dataloaders[phase]:
                  inputs = inputs.to(device)
                  labels = labels.to(device)

                  # zero the parameter gradients
                  optimizer.zero_grad()

                  # forward
                  # track history if only in train
                  with torch.set_grad_enabled(phase == 'train'):
                      outputs = model(inputs)
                      _, preds = torch.max(outputs, 1)
                      loss = criterion(outputs, labels)

                      # backward + optimize only if in training phase
                      if phase == 'train':
                          loss.backward()
                          optimizer.step()

                  # statistics
                  running_loss += loss.item() * inputs.size(0)
                  running_corrects += torch.sum(preds == labels.data)

              epoch_loss = running_loss / dataset_sizes[phase]
              epoch_acc = running_corrects.double() / dataset_sizes[phase]
              print(f"{phase} Running Corrects:,{running_corrects.double()}")
              print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                  phase, epoch_loss, epoch_acc))

              # deep copy the model
              if phase == 'val' and epoch_acc > best_acc:
                  best_acc = epoch_acc
        
                  best_model_wts = copy.deepcopy(model.state_dict())
          print()
    except KeyboardInterrupt:
      print('Interrupting Training...')
      print()
      model.load_state_dict(best_model_wts)  
      if epoch<1:
        return model    

    model.load_state_dict(best_model_wts)  
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    #model.load_state_dict(best_model_wts)
    return model

In [ ]:
def ConfMat(model,data = 'val'):
  since=time.time()
  confusionMatrix=np.zeros((len(class_names),len(class_names)),dtype=int)
  with torch.no_grad():
      for data in dataloaders[data]:
          images, labels = data
          images=images.to(device)
          labels=labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs, 1)
          c = (predicted == labels).squeeze()
          for i in range(4):
            try:
              confusionMatrix[labels[i],predicted[i]]+=1
            except:
              continue
  confusionMatrix=pd.DataFrame(confusionMatrix,columns=['Pred '+class_names[i] for i in range(len(class_names))],index=['Actual '+class_names[i] for i in range(len(class_names))])
  time_elapsed = time.time() - since
  print('{} completed in {:.0f}m {:.0f}s'.format(data,time_elapsed // 60, time_elapsed % 60))
  return confusionMatrix

In [ ]:
import torch.optim as optim
from torch.optim import lr_scheduler


Net = Net.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(Net.parameters(), lr=0.001, momentum=0.9)
#optimizer_ft = optim.SGD(Net.parameters(), lr=0.005, momentum=0.9)
#optimizer_ft = optim.Adam(Net.parameters(),lr=0.0005)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
try:
  del model_ft
except:
  pass

In [ ]:
model_ft = train_model(Net, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_che

train Running Corrects:,4987.0
train Loss: 1.5335 Acc: 0.4656
val Running Corrects:,1554.0
val Loss: 1.1197 Acc: 0.5803

Epoch 1/24
----------
train Running Corrects:,6267.0
train Loss: 1.1479 Acc: 0.5852
val Running Corrects:,1685.0
val Loss: 1.0331 Acc: 0.6292

Epoch 2/24
----------
train Running Corrects:,6687.0
train Loss: 1.0478 Acc: 0.6244
val Running Corrects:,1781.0
val Loss: 0.9292 Acc: 0.6650

Epoch 3/24
----------
train Running Corrects:,6903.0
train Loss: 0.9698 Acc: 0.6445
val Running Corrects:,1774.0
val Loss: 0.9217 Acc: 0.6624

Epoch 4/24
----------
train Running Corrects:,7107.0
train Loss: 0.9221 Acc: 0.6636
val Running Corrects:,1739.0
val Loss: 0.9784 Acc: 0.6494

Epoch 5/24
----------
train Running Corrects:,7317.0
train Loss: 0.8678 Acc: 0.6832
val Running Corrects:,1814.0
val Loss: 0.8978 Acc: 0.6774

Epoch 6/24
----------
train Running Corrects:,8048.0
train Loss: 0.6850 Acc: 0.7514
val Running Corrects:,1910.0
val Loss: 0.7851 Acc: 0.7132

Epoch 7/24
----------

In [ ]:
torch.save(model_ft.state_dict(),'CNN/VGG16_only_fl_BW_Aug.pt')

In [ ]:
model = vgg16
model.load_state_dict(torch.load('CNN/VGG16_only_fl_BW_Aug.pt',map_location=torch.device('cpu')))
model = model.to(device)
model.eval();

In [ ]:
ConfMat(model)

In [ ]:
ConfMat(model,'train')